In [2]:
!pip install "dlt[bigquery]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 11.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.1/456.1 kB 10.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 8.3 MB/s eta 0:00:00


# Question 1

In [ ]:
!dlt --version
# Answer: dlt 1.2.0

dlt 1.2.0


In [4]:
import dlt
print("dlt version:", dlt.__version__)

dlt version: 1.2.0


# Question 2

In [5]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

@dlt.resource(name="rides_dlt", write_disposition="replace")
def ny_taxi_dlt():
    client = RESTClient(
        base_url="https://us-central1-dlthub-analytics.cloudfunctions.net",
        paginator=PageNumberPaginator(
            base_page=1,
            total_path=None
        )
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):
        yield page


pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline_dlt",
    destination="bigquery",
    dataset_name="ny_taxi_data_dlt"
)

load_info = pipeline.run(ny_taxi_dlt)
print(load_info)

/opt/homebrew/Caskroom/miniconda/base/envs/de-zoomcamp/lib/python3.9/site-packages/google/cloud/bigquery/client.py:595: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


Pipeline ny_taxi_pipeline_dlt load step completed in 23.43 seconds
1 load package(s) were loaded to destination bigquery and into dataset ny_taxi_data_dlt
The bigquery destination used de-zoomcamp-warehouse@de-zoomcamp-warehouse.iam.gserviceaccount.com@de-zoomcamp-warehouse location to store data
Load package 1739478619.329885 is LOADED and contains no failed jobs


In [6]:
import os
from google.cloud import bigquery

# Path to your JSON key file
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcs.json"

# Initialize BigQuery client
client = bigquery.Client()

In [4]:
import dlt
pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline_dlt",
    destination="bigquery",
    dataset_name="ny_taxi_data_dlt"
)

In [7]:
# Get the dataset name from the pipeline
dataset_name = pipeline.dataset_name  # Get dataset name from dlt pipeline

# List tables in the dataset
tables = list(client.list_tables(dataset_name))

# Print table names
print(f"Tables in dataset '{dataset_name}':")
for table in tables:
    print(table.table_id)

Tables in dataset 'ny_taxi_data_dlt':
_dlt_loads
_dlt_pipeline_state
_dlt_version
rides_dlt


# Question 3

In [12]:
!pip install db-dtypes

In [8]:
table_name = "rides_dlt"  # The table you want to inspect

# Query to count the records
query = f"SELECT COUNT(*) AS total_records FROM `{dataset_name}.{table_name}`"

# Execute query and convert to DataFrame
df = client.query(query).to_dataframe()

# Display record count
df

/opt/homebrew/Caskroom/miniconda/base/envs/de-zoomcamp/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,total_records
0,10000


# Question 4

In [14]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(TIMESTAMP_DIFF(trip_dropoff_date_time, trip_pickup_date_time, MINUTE))
            FROM rides_dlt;
            """
        )
    # Prints column values of the first row
    print(res)

[Row((12.298199999999992,), {'f0_': 0})]
